In [1]:
#r "nuget: FSharp.Data"
#r "nuget: Plotly.NET"
#r "nuget: Plotly.NET.Interactive"


Installed Packages FSharp.Data, 6.3.0 Plotly.NET, 4.2.0 Plotly.NET.Interactive, 4.2.1

Loading extensions from `C:\Users\bmitc\.nuget\packages\plotly.net.interactive\4.2.1\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [2]:
open FSharp.Data
open Plotly.NET


In [3]:
let text = CsvFile.Load(System.IO.Path.Join([|__SOURCE_DIRECTORY__; "data/pizza.csv"|]), separators=",", hasHeaders=true)

let x,y = [|for row in text.Rows -> (float row[0], float row[1])|] |> Array.unzip


In [4]:
x[0..4]


[ 13, 2, 14, 23, 13 ]

In [5]:
y[0..4]


[ 33, 16, 32, 51, 27 ]

In [6]:
Chart.Point(x, y, Name = "Data")
|> Chart.withXAxisStyle("Reservations")
|> Chart.withYAxisStyle("Pizzas")


<!-- Plotly chart will be drawn inside this DIV -->

In [7]:
let predict (x: float[]) weight = Array.map ((*) weight) x

let average (x: float[]) = (Array.fold (+) 0.0 x) / (float x.Length)

let (.+) (x: float[]) (y: float[]) = Array.map2 (+) x y
let (.-) (x: float[]) (y: float[]) = Array.map2 (-) x y
let (.^) (x: float[]) y = Array.map (fun e -> e ** y) x 

let loss (x: float[]) (y: float[]) weight =
    average(((predict x weight) .- y) .^ 2.0)

let train x y iterations learningRate =
    let mutable w = 0.0
    for i in 0..iterations do
        let currentLoss = loss x y w
        //printfn "Iteration %4d => Loss: %.6f" i currentLoss
        if loss x y (w + learningRate) < currentLoss then
            w <- w + learningRate
        elif loss x y (w - learningRate) < currentLoss then
            w <- w - learningRate
    w


In [8]:
let w = train x y 10000 0.01
printfn "w=%.3f" w
printfn "Prediction: x=%d => y=%.2f" 20 (predict [|20.0|] w).[0]


w=1.840
Prediction: x=20 => y=36.80


In [9]:
let lineX = [| (Array.min x)..1.0..(Array.max y) |]
let lineY = predict lineX w

Chart.Line(Array.zip lineX lineY, Name = "Prediction")
|> Chart.withXAxisStyle "Reservations"
|> Chart.withYAxisStyle "Pizzas"


<!-- Plotly chart will be drawn inside this DIV -->

In [10]:
let predict2 (x: float[]) weight bias = Array.map (fun e -> e*weight + bias) x

let loss2 (x: float[]) (y: float[]) weight bias =
    average(((predict2 x weight bias) .- y) .^ 2.0)

let train2 x y iterations learningRate =
    let mutable weight = 0.0
    let mutable bias = 0.0
    for i in 0..iterations do
        let currentLoss = loss2 x y weight bias
        if loss2 x y (weight + learningRate) bias < currentLoss then
            weight <- weight + learningRate
        elif loss2 x y (weight - learningRate) bias < currentLoss then
            weight <- weight - learningRate
        elif loss2 x y weight (bias + learningRate) < currentLoss then
            bias <- bias + learningRate
        elif loss2 x y weight (bias - learningRate) < currentLoss then
            bias <- bias - learningRate
    weight, bias


In [11]:
let w, b = train2 x y 10000 0.01

printfn "w=%.3f, b=%.3f" w b
printfn "Prediction: x=%d => y=%.2f" 20 (predict2 [|20.0|] w b).[0]


w=1.100, b=12.930
Prediction: x=20 => y=34.93


In [12]:
let lineX = [| (Array.min x)..1.0..(Array.max x) |]
let lineY = predict2 lineX w b

[ Chart.Point(x, y, Name = "Data");
  Chart.Line(Array.zip lineX lineY, Name = "Prediction") ]
|> Chart.combine
|> Chart.withXAxisStyle "Reservations"
|> Chart.withYAxisStyle "Pizzas"


<!-- Plotly chart will be drawn inside this DIV -->